In [1]:
import cv2
import imutils
import numpy as np
import argparse

In [2]:
HOGCV = cv2.HOGDescriptor()
HOGCV.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

In [3]:
def detect(frame):
    bounding_box_cordinates, weights =  HOGCV.detectMultiScale(frame, winStride = (4, 4), padding = (8, 8), scale = 1.03)
    
    person = 1
    for x,y,w,h in bounding_box_cordinates:
        cv2.rectangle(frame, (x,y), (x+w,y+h), (0,255,0), 2)
        cv2.putText(frame, f'person {person}', (x,y), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1)
        person += 1
    
    cv2.putText(frame, 'Status : Detecting ', (40,40), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255,0,0), 2)
    cv2.putText(frame, f'Total Persons : {person-1}', (40,70), cv2.FONT_HERSHEY_DUPLEX, 0.8, (255,0,0), 2)
    cv2.imshow('output', frame)

    return frame

# Normal Human Detection (Camera)

In [ ]:
import cv2
import imutils

HOGCV = cv2.HOGDescriptor()
HOGCV.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

def detect(frame):
    # detect people in the image
    bounding_box_cordinates, weights = HOGCV.detectMultiScale(
        frame,
        winStride=(4, 4),
        padding=(8, 8),
        scale=1.03
    )
    person = 1
    for (x, y, w, h) in bounding_box_cordinates:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, f'person {person}', (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
        person += 1

    cv2.putText(frame, 'Status : Detecting ', (40, 40),
                cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 0, 0), 2)
    cv2.putText(frame, f'Total Persons : {person - 1}', (40, 70),
                cv2.FONT_HERSHEY_DUPLEX, 0.8, (255, 0, 0), 2)

    cv2.imshow("Human Detection", frame)
    return frame

def humanDetector(video_path):
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = imutils.resize(frame, width=min(800, frame.shape[1]))
        detect(frame)
        key = cv2.waitKey(1) & 0xFF
        if key == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    video_path = r"YOUR_VIDEO_PATH" # Paste your video path here
    humanDetector(video_path)


# Optimized FPS + Added Novelty of Entry-Exit Count

In [ ]:
import cv2
import numpy as np
import math

hog = cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())

trackers = []
track_paths = {}
track_colors = {}
track_ids = []
entry_count = 0
exit_count = 0
current_id = 0

line_position = 320
DIST_THRESHOLD = 50  # Matching threshold

def distance(p1, p2):
    return math.hypot(p1[0] - p2[0], p1[1] - p2[1])

def crossed_the_line(trail, line_y=line_position):
    if len(trail) < 2:
        return None
    y_prev = trail[-2][1]
    y_now = trail[-1][1]
    if y_prev < line_y and y_now >= line_y:
        return "entry"
    elif y_prev > line_y and y_now <= line_y:
        return "exit"
    return None

def detect_people(frame):
    (rects, _) = hog.detectMultiScale(
        frame, winStride=(4, 4), padding=(8, 8), scale=1.05)
    return [(x, y, w, h) for (x, y, w, h) in rects]

def init_tracker(tracker_type="MOSSE"):
    try:
        if tracker_type == "KCF":
            return cv2.legacy.TrackerKCF_create()
        elif tracker_type == "CSRT":
            return cv2.legacy.TrackerCSRT_create()
        else:
            return cv2.legacy.TrackerMOSSE_create()
    except AttributeError:
        if tracker_type == "KCF":
            return cv2.TrackerKCF_create()
        elif tracker_type == "CSRT":
            return cv2.TrackerCSRT_create()
        else:
            return cv2.TrackerMOSSE_create()

def main(video_path):
    global current_id, entry_count, exit_count

    cap = cv2.VideoCapture(video_path)
    frame_count = 0

    # Output video writer initialization
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    out = cv2.VideoWriter('output_detected.mp4', fourcc, fps, (800, 640))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.resize(frame, (800, 640))

        if frame_count % 60 == 0:
            detected = detect_people(frame)
            new_trackers = []
            for (x, y, w, h) in detected:
                cx, cy = int(x + w / 2), int(y + h / 2)
                matched = False
                for i, tid in enumerate(track_ids):
                    trail = track_paths.get(tid, [])
                    if trail:
                        last_pos = trail[-1]
                        if distance((cx, cy), last_pos) < DIST_THRESHOLD:
                            matched = True
                            break
                if not matched:
                    tracker = init_tracker()
                    tracker.init(frame, (x, y, w, h))
                    new_trackers.append(tracker)
                    track_ids.append(current_id)
                    track_paths[current_id] = [(cx, cy)]
                    track_colors[current_id] = (
                        np.random.randint(0, 255),
                        np.random.randint(0, 255),
                        np.random.randint(0, 255)
                    )
                    current_id += 1
            trackers.extend(new_trackers)

        updated_trackers = []
        new_ids = []
        for i, tracker in enumerate(trackers):
            success, box = tracker.update(frame)
            if not success:
                continue
            x, y, w, h = [int(v) for v in box]
            cx, cy = int(x + w / 2), int(y + h / 2)
            tid = track_ids[i]
            trail = track_paths[tid]
            trail.append((cx, cy))
            if len(trail) > 20:
                trail = trail[-20:]
            track_paths[tid] = trail
            color = track_colors[tid]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv2.putText(frame, f'ID {tid}', (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
            for j in range(1, len(trail)):
                cv2.line(frame, trail[j - 1], trail[j], color, 2)

            status = crossed_the_line(trail)
            if status == "entry":
                entry_count += 1
                track_paths[tid] = []
            elif status == "exit":
                exit_count += 1
                track_paths[tid] = []

            updated_trackers.append(tracker)
            new_ids.append(tid)

        trackers[:] = updated_trackers
        track_ids[:] = new_ids

        cv2.line(frame, (0, line_position), (frame.shape[1], line_position), (0, 0, 255), 2)
        cv2.putText(frame, f"Entries: {entry_count}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(frame, f"Exits: {exit_count}", (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

        cv2.imshow("Optimized Human Tracker", frame)
        out.write(frame)  # <-- Save processed frame

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_count += 1

    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    video_path = r"YOUR_VIDEO_PATH" # Paste your video path here
    main(video_path)
